# Simple control file editor

This notebook is used to update control files for Calypso. The first (this) section contains programs to load, edit, and store control files. The second section is the description of control items. In Jupyter Lab, you can display the control parameter descriptions and control block editor side by side, and edit control data by looking at the descritions of the control parameters.

The follwing routines are used in this editor:

- `[Instance] = Control_Obj()`  
Initialize class for control data named [Instance].
- `[Instance].new_control_data([block_names])`  
Create new control data with empty block named [block_names] to [Instance].
- `[Instance].read_control_file([file_name])`  
Read and store control data from [file_name] to [Instance].
- `[Instance].write_control_file([file_name])`  
Write control data in [Instance].
- `[List_Instance] = Show_Block_Names([Instance], [Block_names])`  
List block and external control file names.
- `[Widget_instance] = Control_Block_Editor([Instance], [Block_names])`  
Open a editor for control block or array including special editors.

## Prepare functions to edit control blocks
The preparation consists of five steps. The first is loading required libraries, and the next step is loading class to store and edit control data.

#### Loading libraries

Load libraries for this notebook are loaded.  
- os  
Interface to operation systems. It is one of the standard libraries.
- re  
Treat regular expression. It is one of the standard libraries.

- IPython  
A rich architecture for interactive computing (https://ipython.org)
- ipywidets  
Interactive HTML widgets for Jupyter notebooks and the IPython kernel. (https://pypi.org/project/ipywidgets/)

In [1]:
import os
import re
import IPython
import ipywidgets as widgets
from IPython.display import Math, display

IPython.display.HTML('<style> select, textarea, input {font-family: Courier new; } <\style>')

#### Routines 1 (control data IO)

Make class `Control_Obj` to treat control file.  
Following routines are used:

- `[Instance] = Control_Obj()` 
Initialize class for control data named [Instance].
- `[Instance].new_control_data([block_names])` 
Create new control data with empty block named [block_names] to [Instance].
- `[Instance].read_control_file([file_name])` 
Read and store control data from [file_name] to [Instance].
- `[Instance].write_control_file([file_name])` 
Write control data in [Instance].



In [2]:
class Control_Obj(object):
    def __init__(self):
        self.file_name = ""
        self.datalist = [""]
        self.datalist.clear()
            
    def read_control_file(self, file_name):
        self.file_name = file_name
        f = open(file_name, 'r')
        self.datalist = f.readlines()
        f.close()
        print('File \'', file_name, '\' is loaded.')
        print(file_name, ' has ', len(self.datalist), 'lines.')

    def write_control_file(self, file_name):
        if(os.path.isfile(file_name)):
            print("File ", file_name, "exists. No overwrite")
        else:
            f = open(file_name, 'w')
            f.writelines(self.datalist)
            f.close()
            print("File ", file_name, "is written.")
    
    def new_control_data(self, block_name):
        self.datalist.append('begin ' + block_name + '\n')
        self.datalist.append('end ' + block_name + '\n')
        print('******* New control data  *********')

        for index in range(len(self.datalist)):
            print(index, self.datalist[index], end='')

#### Routines 2 (Search block)

Make class `Block_Finder` to find start and end lines of block. All subroutines are used internally.

In [3]:
class Block_Finder(object):
    def find_target_block_name(self, block_names):
        block_list = block_names.split(None)
        if(len(block_list) == 0):
            return ""
        return block_list[len(block_list)-1];
    
    def __count_block_w_range(self, datalist, block_name, outside_block):
        icou = 0
        for index in range(outside_block[0], outside_block[1]):
            if(len(datalist[index].split(None)) < 2):
                continue
            block_label = datalist[index].split(None)[1]
            if(block_name.upper() == block_label.upper()):
                if(re.match('^\s*file',datalist[index])):
                    icou = icou + 1
                if(re.match('^\s*begin',datalist[index])):
                    icou = icou + 1
        if(icou > 1):
            print("There is more than one block with same name between")
            block_label = datalist[outside_block[0]]
            print("    \'", outside_block[0], ": ", block_label.rstrip(), "\'")
            block_label = datalist[outside_block[1]-1]
            print("and \'", outside_block[1], ": ", block_label.rstrip(), "\'.")
        return icou

    def begin_and_end_block_w_range(self, datalist, block_name, outside_block):
        array_index = -1
        if("[" in block_name):
            stripped = re.sub("\[.*\]", "", block_name)
            tmp_num = re.sub(".*\[", "", block_name)
            array_index = int(re.sub("\]", "", tmp_num))
            icou = 0
        else:
            stripped = block_name
        
        blk_line = [-1,-1]
        if(array_index == -1):
            if(self.__count_block_w_range(datalist, stripped, outside_block) > 1):
                print("""Specify index of array block by \[\#\] 
                       or add belonged block in the block name list""")
                return blk_line
        
        for index in range(outside_block[0], outside_block[1]):
            if(len(datalist[index].split(None)) < 2):
                continue
            block_label = datalist[index].split(None)[1]
            if(stripped.upper() == block_label.upper()):
                if(re.match('^\s*file',datalist[index]) \
                  and array_index == -1):
                    blk_line[0] = index
                    blk_line[1] = index
                    print(stripped, 'is in the external file ', \
                         datalist[index].split(None)[2])
                    print('Please load control data from \'', end="")
                    print(datalist[index].split(None)[2], end="")
                    print('\' into new control data instance by ')
                    print(' [New_instance] = Control_Obj()')
                    print(' [New_instance].read_control_file(\'', \
                          datalist[index].split(None)[2], '\')\n')
                    break
#   Find starting of block
                elif(re.match('^\s*begin',datalist[index]) \
                  and array_index == -1):
                    blk_line[0] = index
#   Find starting of array block
                elif(re.match('^\s*array',datalist[index]) \
                  and array_index == 0):
                    blk_line[0] = index
#   Find starting of block in array
                elif(re.match('^\s*begin',datalist[index]) \
                  and array_index > 0):
                    icou = icou + 1
                    if(icou == array_index):
                        blk_line[0] = index
                        
        if(blk_line[0] < 0):
            if(array_index == 0):
                print("Array", stripped, " is missing.")
            elif(array_index == -1):
                print("Block", stripped, " is missing.")
            else:
                print(array_index, "-th Block", stripped, " is missing.")
            return blk_line

        for index in range(blk_line[0], outside_block[1]):
            if(len(datalist[index].split(None)) < 2):
                continue
#   Find end of array block
            if(re.match('^\s*end\s*array',datalist[index]) \
              and array_index == 0):
                block_label = datalist[index].split(None)[2]
                if(stripped.upper() == block_label.upper()):
                    blk_line[1] = index
                    break
#   Find starting of end block
            elif(re.match('^\s*end',datalist[index]) \
              and array_index != 0):
                block_label = datalist[index].split(None)[1]
                if(stripped.upper() == block_label.upper()):
                    blk_line[1] = index
                    break

        if(blk_line[1] < 0):
            if(array_index == 0):
                print("Array", stripped, " is not closed.")
            elif(array_index == -1):
                print("Block", stripped, " is not closed.")
            else:
                print(array_index, "-th Block", stripped, " is not closed.")
        return blk_line

    def begin_and_end_block(self, datalist, block_names):
        block_list = block_names.split(None)
        num_blk_name = len(block_list)
        blk_line = [0, len(datalist)]

        for index in range(len(block_list)):        
            blk_line = self.begin_and_end_block_w_range(datalist, block_list[index], blk_line)
        return blk_line

#### Routines 3 (List control blocks)

Make class `Show_Block_Names` to show loaded control data.

- `[List_Instance] = Show_Block_Names([Instance], [Block_name])`

This command lists control, file, and array blocks below `[Block_name]` in the control file data [Instance] which is read from control file. The output is a list of  
- Line #, lebel of block, and text including the block name.  

This routine also send a message if blokes does not have consistency. 

In [4]:
class Show_Block_Names(Block_Finder):
    def __init__(self, control, top_block):
        self.block_list(control.datalist, top_block)

    def __find_blocks(self, datalist, level, blk_list, blk_line):
        if(blk_line[0]+1 > blk_line[1]):
            return level
    
        for index in range(blk_line[0]+1, blk_line[1]+1):
            if(index >= len(datalist)):
                break
            if(re.match('^\s*file',datalist[index])):
                print("{:>4d} {:>2d}".format(index, level+1), datalist[index], end="")
    
            if(re.match('^\s*end\s*array',datalist[index])):
                current_blk = blk_list[level-1]
                blk_name = datalist[index].split(None)[2]
                if(blk_name.upper() == current_blk.upper()):
                    level = level - 1
#                    print('end array', level)
                    del blk_list[level]
            
            elif(re.match('^\s*end',datalist[index])):
                current_blk = blk_list[level-1]
                blk_name = datalist[index].split(None)[1]
                if(blk_name.upper() == current_blk.upper()):
                    level = level - 1
#                    print('end block', level)
                    del blk_list[level]
    
            elif(re.match('^\s*begin',datalist[index])):
                child_blk = datalist[index].split(None)[1]
                blk_list.append(child_blk)
                level = level + 1
                blk_line[0] = index
                print("{:>4d} {:>2d}".format(index, level), datalist[index], end="")
                return 1 + self.__find_blocks(datalist, level, blk_list, blk_line)
    
            elif(re.match('^\s*array',datalist[index])):
                child_blk = datalist[index].split(None)[1]
                blk_list.append(child_blk)
                level = level + 1
                blk_line[0] = index
                print("{:>4d} {:>2d}".format(index, level), datalist[index], end="")
                return 1 + self.__find_blocks(datalist, level, blk_list, blk_line)
    
        if(level != 0):
            print("control blocks does not close correctory. Need checks.", level)
        return level

    def block_list(self, datalist, top_block):
        level = 0
        blk_line = self.begin_and_end_block(datalist, top_block)
        print("Line#, block level, text" )
        for index in range(blk_line[0], blk_line[1]):
            if(re.match('^\s*begin',datalist[index])):
                level = level + 1
                block_name = datalist[index].split(None)[1]
                blk_list = [block_name]
                blk_line[0] = index
                print("{:>4d} {:>2d}".format(index, level), blk_line, datalist[index], end="")
                
                level_out = self.__find_blocks(datalist, level, blk_list, blk_line)
                break
        return

#### Routines 4 (block editor)

Make class `Control_Block_Editor` to make block Editor from `Block_Finder`. Block editer is called by

- `[Widget_instance] = Control_Block_Editor([Instance], [Block_name])`

This command opens a text editor for block [Block_name] in the control file data [Instance] which is read from control file. After editind the block, update is applied when Update [Block_name] bottun is clicked.  The updated control data around the block are also displayed after clicking the bottun. [Widget_instance] name is only used to make the editor.

In the cases when there is more than one block with the same name, `[Block_name]` can specify the belonged blocks sepalated by space. For example, the folloing command opens editor for `phys_values_ctl` in `model` in `MHD_control`.

`a = Control_Block_Editor(ctl, 'MHD_control model phys_values_ctl')`

If the target block is the array block, Add '[0]' after the array block name. For example, the folloing command opens editor for array `volume_spectrum_ctl` in `sph_monitor_ctl`.

`b = Control_Block_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[0]')`

To specify i-th (i start from 1) block in array block set [i] after the block name. For example, the folloing command opens editor for 2nd block of `volume_spectrum_ctl` in `sph_monitor_ctl`.

`c = Control_Block_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[2]')`



In [5]:
class Control_Block_Editor(Block_Finder):
    def  __init__(self, control, block_names):
        self.datalist = control.datalist
        self.update_output = widgets.Output()
        self.blk_names = block_names
        self.block_editbox()
#        print('control.datalist', id(control.datalist))
#        print('self.datalist init', id(self.datalist))



    def make_editbox(self, blk_names):
        blk_line = self.begin_and_end_block(self.datalist, blk_names)

        block_text = ''
        for index in range(blk_line[0],blk_line[1]+1):
            block_text = block_text + self.datalist[index]

        Tx = widgets.Textarea(
            value=block_text,
            placeholder='Type something',
            description='',
            disabled=False,
            rows=(blk_line[1] - blk_line[0] + 2),                            
            layout=widgets.Layout(width="auto")
            )
        return Tx
            
    def make_update_button(self, blk_names):
        block_name = self.find_target_block_name(blk_names)
        b_layout = widgets.Layout(width='auto', height='40px')
        update_button = widgets.Button(description=("Update " + block_name),
                                       button_style='success',
                                       layout = b_layout)
        update_button.on_click(self.update_button_pressed)
        return update_button

    def make_clear_button(self):
        b_layout = widgets.Layout(width='auto', height='40px')
        update_button = widgets.Button(description=("Clear Log"),
                                       button_style='success',
                                       layout = b_layout)
        update_button.on_click(self.clear_button_pressed)
        return update_button


    def block_editbox(self):
        self.Tx =            self.make_editbox(self.blk_names)
        self.update_button = self.make_update_button(self.blk_names)
        print("To reload current block, press \"shift\" + \"return\".")
        display(self.Tx, self.update_button, self.update_output)
    
    def update_block(self):
        self.update_output.clear_output()
        
        new_datalist = self.Tx.value.splitlines(True)
        new_nline = len(new_datalist)
        blk_line = self.begin_and_end_block(self.datalist, self.blk_names)

#        print("Old end Position ", blk_line[1], self.datalist[blk_line[1]])

        for index in range(blk_line[0],blk_line[1]+1):
            del self.datalist[blk_line[0]]
        for index in range(new_nline):
            self.datalist.insert(blk_line[0]+index, new_datalist[index])
        blk_line[1] = blk_line[0]+new_nline-1
#        print("New end Position ", blk_line[1], self.datalist[blk_line[1]])
        if(blk_line[0]<2):
            istart = 0
        else:
            istart = blk_line[0]-2
        
        if(blk_line[1]+3>len(self.datalist)-1):
            iend = len(self.datalist)
        else:
            iend = blk_line[1]+3

        with self.update_output:
#            print('self.datalist updated', id(self.datalist))

            self.clear_button = self.make_clear_button()
            display(self.clear_button)
            print('************   Updated block   **************')
            for index in range(istart, iend):
                print(index, self.datalist[index], end='')


    def update_button_pressed(self,*args):
        self.update_block()
    def clear_button_pressed(self,*args):
        self.update_output.clear_output()
   

#### Routins 5 (Special block editors)

Make classes for special editor box for governing equation.

The following routines are for this class:
- `[Widget_instance] = Dimless_ctl_Editor([Instance])`  
Open editor for dimensionless numbers in [Instance].
- `[Widget_instance] = Momentum_Eq_ctl_Editor([Instance])`  
Open editor for coefficients of momentum equation in [Instance].
- `[Widget_instance] = Induction_Eq_ctl_Editor([Instance])`  
Open editor for coefficients of heat equation in [Instance].
- `[Widget_instance] = Heat_Eq_ctl_Editor([Instance])`  
Open editor for coefficients of composition equation in [Instance].
- `[Widget_instance] = Composition_Eq_ctl_Editor([Instance])`  
Open editor for coefficients of induction equation in [Instance].

- `[Widget_instance] = Four_eqs_Editor([Instance])`  
Open editor for the all goverining equations in [Instance].

- `[Widget_instance] = Four_eqs_Editor([Instance])`  
Open editor for the all goverining equations in [Instance].

- `[Widget_instance] = Surface_define_editor([Instance], [Block_name])`  
Open editor for cross section confiurations in [Instance].

In [6]:

class Dimless_ctl_Editor(Control_Block_Editor):
    def  __init__(self, control):
        self.datalist = control.datalist
        self.update_output = widgets.Output()
        self.dimless_output = widgets.Output()

        self.blk_names = 'MHD_control model dimensionless_ctl dimless_ctl[0]'

        self.dimensionless_editbox()

    def check_coef_block(self, block_names):
        self.blk_line = self.begin_and_end_block(self.datalist, block_names)
        if(self.blk_line[0] < 0):
            print(self.find_target_block_name(block_names) + 'is not found.')
            print('Make under \'coefficients_ctl\' block.')
            return 1
        return 0

    def print_dimless_list(self):
        block_names = 'MHD_control model dimensionless_ctl dimless_ctl[0]'
        tgt_label = 'dimless_ctl'
        dimless_line = self.begin_and_end_block(self.datalist, block_names)
        with self.dimless_output:
            print('*******  Avaiable numbers and current value   *********')
            print('     Zero = 0.0')
            print('     One  = 1.0')
            print('     Two  = 2.0')
            print('     Radial_35 = 0.65')
            for index in range(dimless_line[0]+1,dimless_line[1]):
                linetext = self.datalist[index]
                label = linetext.split(None)[0]
                if(label.upper() == tgt_label.upper()):
                    print('     ' + linetext.split(None)[1] +' = ' \
                          + linetext.split(None)[2])
            print("\n To reload current block, press \"shift\" + \"return\".")

    def const_coefs(self, array_name, math_term):
        line_range = self.begin_and_end_block_w_range(self.datalist, array_name, self.blk_line)
        coef_text = ""
        for index in range(line_range[0]+1,line_range[1]):
            linetext = self.datalist[index]
            tmpcoef = linetext.split(None)[1]
            coef_text = coef_text + "( {\\tt " + tmpcoef.replace("_", "\_")
            coef_text = coef_text + "})^{" + linetext.split(None)[2] + "} "
        if(len(coef_text) > 0 ):
            coef_text = coef_text + math_term
        return coef_text
   
    def dimensionless_editbox(self):
        self.block_names = 'MHD_control model dimensionless_ctl dimless_ctl[0]'
        if(self.check_coef_block(self.blk_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.dless_button = widgets.Button(description="Update Equation",
                                           button_style='success')
        self.dless_button.on_click(self.update_dimless_pressed)
        display(self.dimless_output, self.Tx, self.dless_button, self.update_output)

    def update_dimless(self):
        self.dimless_output.clear_output()
        self.print_dimless_list()
 
    def update_dimless_pressed(self,*args):
        self.update_block()
        self.update_dimless()


class Momentum_Eq_ctl_Editor(Dimless_ctl_Editor):
    def  __init__(self, control):
        self.datalist = control.datalist
        self.mom_output = widgets.Output()
        self.update_output = widgets.Output()
        self.dimless_output = widgets.Output()

        self.blk_names = 'MHD_control model coefficients_ctl momentum'

        self.momentum_eq_editbox()

    def __mom_equation_text(self):
        mom_text = self.const_coefs('coef_4_velocity_ctl[0]', \
                                     '\\left( \\frac{D {\\bf u}}{D t} \\right)')
        prs_text = self.const_coefs('coef_4_press_ctl[0]', \
                                     ' \\nabla P')
        vis_text = self.const_coefs('coef_4_v_diffuse_ctl[0]', \
                                     '\\nabla^{2} {\\bf u}')
        tbo_text = self.const_coefs('coef_4_buoyancy_ctl[0]', \
                                     '(T {\\bf r})')
        cbo_text = self.const_coefs('coef_4_composit_buoyancy_ctl[0]', \
                                     '(C {\\bf r})')
        cor_text = self.const_coefs('coef_4_Coriolis_ctl[0]', \
                                     '(\\hat{z} \\times {\\bf u})')
        lor_text = self.const_coefs('coef_4_Lorentz_ctl[0]', \
                                     '( (\\nabla \\times {\\bf B}) \\times {\\bf B})')
        full_text = ["",""]
        full_text[0] = mom_text
        if(len(prs_text) > 0):
            full_text[0] = full_text[0] + ' = - ' + prs_text
        if(len(vis_text) > 0):
            full_text[0] = full_text[0] + ' + ' + vis_text
        if(len(cor_text) > 0):
            full_text[0] = full_text[0] + ' - ' + cor_text
       
        full_text[1] = '\\ \\ \\ \\ \\ \\ \\ \\ '
        if(len(tbo_text) > 0):
            full_text[1] = full_text[1] + ' + ' + tbo_text
        if(len(cbo_text) > 0):
            full_text[1] = full_text[1] + ' + ' + cbo_text
        if(len(lor_text) > 0):
            full_text[1] = full_text[1] + ' + ' + lor_text
        return full_text


    def momentum_eq_editbox(self):
        if(self.check_coef_block(self.blk_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.math_button = widgets.Button(description="Update Equation",
                                           button_style='success')
        self.math_button.on_click(self.update_mom_eq_pressed)
 
        display(self.mom_output, self.dimless_output, self.Tx,
                self.math_button, self.update_output)
        self.update_mom_equation()
        
    def update_mom_equation(self):
        full_text = self.__mom_equation_text()

        self.dimless_output.clear_output()
        self.mom_output.clear_output()
        self.print_dimless_list()
        with self.mom_output:
            print('************  Momeuntum equation  **************')
            display(Math(full_text[0]))
            display(Math(full_text[1]))        

    def update_mom_eq_pressed(self,*args):
        self.update_block()
        self.update_mom_equation()


class Induction_Eq_ctl_Editor(Dimless_ctl_Editor):
    def  __init__(self, control):
        self.datalist = control.datalist
        self.mag_output = widgets.Output()
        self.update_output = widgets.Output()
        self.dimless_output = widgets.Output()

        self.blk_names = 'MHD_control model coefficients_ctl induction'

        self.induction_eq_editbox()

    def __induction_equation_text(self):
        mag_text = self.const_coefs('coef_4_magnetic_ctl[0]', \
                                     '\\left( \\frac{\\partial {\\bf B}}{\\partial t} \\right)')
        dif_text = self.const_coefs('coef_4_m_diffuse_ctl[0]', \
                                     '\\nabla^{2} {\\bf B}')
        uxb_text = self.const_coefs('coef_4_induction_ctl[0]', \
                                     '({\\bf u} \\times {\\bf B})')
        full_text = mag_text
        if(len(dif_text) > 0):
            full_text = full_text + ' = ' + dif_text
        if(len(uxb_text) > 0):
            full_text = full_text + ' + ' + uxb_text
        return full_text

    def induction_eq_editbox(self):
        block_names = 'MHD_control model coefficients_ctl induction'
        if(self.check_coef_block(block_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.math_button = widgets.Button(description="Update Equation",
                                           button_style='success')
        self.math_button.on_click(self.update_induct_eq_pressed)
        display(self.mag_output, self.dimless_output, self.Tx,
                self.math_button, self.update_output)
        self.update_induction_equation()
        
    def update_induction_equation(self):
        full_text = self.__induction_equation_text()

        self.mag_output.clear_output()
        self.dimless_output.clear_output()
        
        self.print_dimless_list()
        with self.mag_output:
            print('************  Induction equation  **************')
            display(Math(full_text))

    def update_induct_eq_pressed(self,*args):
        self.update_block()
        self.update_induction_equation()



class Heat_Eq_ctl_Editor(Dimless_ctl_Editor):
    def  __init__(self, control):
        self.datalist = control.datalist
        self.heat_output = widgets.Output()
        self.update_output = widgets.Output()
        self.dimless_output = widgets.Output()

        self.blk_names = 'MHD_control model coefficients_ctl thermal'
        self.heat_eq_editbox()

    def __heat_equation_text(self):
        het_text = self.const_coefs('coef_4_termal_ctl[0]', \
                                     '\\left( \\frac{D T}{D t} \\right)')
        dif_text = self.const_coefs('coef_4_t_diffuse_ctl[0]', \
                                     ' \\nabla^{2} T')
        src_text = self.const_coefs('coef_4_heat_source_ctl[0]', \
                                     'Q_{T}')
        full_text = het_text
        if(len(dif_text) > 0):
            full_text = full_text + ' = ' + dif_text
        if(len(src_text) > 0):
            full_text = full_text + ' + ' + src_text
        return full_text

    def heat_eq_editbox(self):
        if(self.check_coef_block(self.blk_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.math_button = widgets.Button(description="Update Equation",
                                           button_style='success')
        self.math_button.on_click(self.update_heat_eq_pressed)
        display(self.heat_output, self.dimless_output,
                self.Tx, self.math_button, self.update_output)
        self.update_heat_equation()

    def update_heat_equation(self):
        full_text = self.__heat_equation_text()

        self.dimless_output.clear_output()
        self.heat_output.clear_output()
        
        self.print_dimless_list()
        with self.heat_output:
            print('************  Heat equation  **************')
            display(Math(full_text))

    def update_heat_eq_pressed(self,*args):
        self.update_block()
        self.update_heat_equation()
    

class Composition_Eq_ctl_Editor(Dimless_ctl_Editor):
    def  __init__(self, control):
        self.datalist = control.datalist
        self.comp_output = widgets.Output()
        self.update_output = widgets.Output()
        self.dimless_output = widgets.Output()

        self.blk_names = 'MHD_control model coefficients_ctl composition'
        self.composition_eq_editbox()

    def __composition_equation_text(self):
        cmp_text = self.const_coefs('coef_4_composition_ctl[0]', \
                                     '\\left( \\frac{D C}{D t} \\right)')
        dif_text = self.const_coefs('coef_4_c_diffuse_ctl[0]', \
                                     ' \\nabla^{2} C')
        src_text = self.const_coefs('coef_4_light_source_ctl[0]', \
                                     'Q_{C}')
        full_text = cmp_text
        if(len(dif_text) > 0):
            full_text = full_text + ' = + ' + dif_text
        if(len(src_text) > 0):
            full_text = full_text + ' + ' + src_text
        return full_text

    def composition_eq_editbox(self):
        if(self.check_coef_block(self.blk_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.math_button = widgets.Button(description="Update Equation",
                                           button_style='success')
        self.math_button.on_click(self.update_comp_eq_pressed)

        display(self.comp_output, self.dimless_output, 
                self.Tx, self.math_button, self.update_output)
        self.update_comp_equation()
        

    def update_comp_equation(self):
        full_text = self.__composition_equation_text()
        
        self.dimless_output.clear_output()
        self.comp_output.clear_output()
        
        self.print_dimless_list()
        with self.comp_output:
            print('************  Conposition equation  **************')
            display(Math(full_text))
        
    def update_comp_eq_pressed(self,*args):
        self.update_block()
        self.update_comp_equation()

        
class Four_eqs_Editor(Momentum_Eq_ctl_Editor, Induction_Eq_ctl_Editor, 
                     Heat_Eq_ctl_Editor, Composition_Eq_ctl_Editor):
    def __init__(self, control):
        self.datalist = control.datalist
        self.blk_names = 'MHD_control model coefficients_ctl'
        self.update_output = widgets.Output()
        self.heat_output = widgets.Output()
        self.comp_output = widgets.Output()
        self.mom_output = widgets.Output()
        self.mag_output = widgets.Output()

        self.dimless_output = widgets.Output()

        self.four_eq_editbox()
    
    def four_eq_editbox(self):
        if(self.check_coef_block(self.blk_names) > 0):
            return
        self.print_dimless_list()
        self.Tx = self.make_editbox(self.blk_names)

        self.math_button = widgets.Button(description="Update Equations",
                                           button_style='success')
        self.math_button.on_click(self.update_gov_eq_pressed)


        display(self.mom_output, self.mag_output, 
                self.heat_output, self.comp_output,
                self.dimless_output, self.Tx, self.math_button,
                self.update_output)
        self.update_mom_equation()
        self.update_induction_equation()
        self.update_heat_equation()
        self.update_comp_equation()
    
    def update_gov_eq_pressed(self,*args):
        self.update_block()
        self.update_mom_equation()
        self.update_induction_equation()
        self.update_heat_equation()
        self.update_comp_equation()



In [7]:
class Surface_define_editor(Control_Block_Editor):
    math_v = ['x', 'y', 'z']

    def  __init__(self, control, block_names):
        self.datalist = control.datalist
        self.surface_eq_output = widgets.Output()
        self.update_output = widgets.Output()
        self.blk_names = block_names
        
        self.surface_coef_editbox()
        
    def __surface_equation_text(self):
        math_q = ['X2','Y2','Z2','YZ','ZX','XY','X','Y','Z','CONST']
        math_t = ['x^{2}','y^{2}','z^{2}','yz','zx','xy','x','y','z']
        coef_txt = ['0.0']*10
        coef_blk_name = self.blk_names + '  coefs_ctl[0]'
        line_range = self.begin_and_end_block(self.datalist, coef_blk_name)
        for index in range(line_range[0]+1,line_range[1]):
            linetext = self.datalist[index]
            if(len(linetext) < 3):
                continue

            tmp_dir = linetext.split(None)[1]
            tmpcoef = linetext.split(None)[2]

            for idir in range(len(math_q)):
                if(tmp_dir.upper() == math_q[idir].upper()):
                    coef_txt[idir] = tmpcoef
                    continue
                    
        surf_math = ""
        for idir in range(9):
            if(float(coef_txt[idir]) != 0.0):
                surf_math = surf_math + '(' + coef_txt[idir] + ')' + math_t[idir] + ' + ' 
        surf_math = surf_math + '('+ coef_txt[9] + ') = 0'
        return surf_math

    def __position_text(self, center_blk_name):
        center_txt = ['0.0']*3
        center_range = self.begin_and_end_block(self.datalist, center_blk_name)
        for index in range(center_range[0]+1,center_range[1]):
            linetext = self.datalist[index]
            if(len(linetext) < 3):
                continue

            tmp_dir = linetext.split(None)[1]
            tmpcoef = linetext.split(None)[2]
            for idir in range(len(self.math_v)):
                if(tmp_dir.upper() == self.math_v[idir].upper()):
                    center_txt[idir] = tmpcoef
                    continue
        return center_txt

    def __sphere_equation_text(self):
        radius_label = 'radius'
        center_blk_name = self.blk_names + '  center_position[0]'
        center_txt = self.__position_text(center_blk_name)

        line_range = self.begin_and_end_block(self.datalist, self.blk_names)
        for index in range(line_range[0]+1,line_range[1]):
            linetext = self.datalist[index]
            if(len(linetext) < 2):
                continue

            label = linetext.split(None)[0]
            if(label.upper() == radius_label.upper()):
                radius_txt = linetext.split(None)[1]
                continue

                
        surf_math = ""
        for idir in range(len(self.math_v)):
            if(float(center_txt[idir]) == 0.0):
                surf_math = surf_math + self.math_v[idir] + '^{2}'
            else:
                surf_math = surf_math + '[' + self.math_v[idir] + ' - (' + center_txt[idir] + ')]^{2}'

            if(idir == 2):
                surf_math = surf_math + ' = (' + radius_txt + ')^{2}'
            else:
                surf_math = surf_math + ' + '
        return surf_math


    def __plane_equation_text(self):
        center_blk_name = self.blk_names + '  center_position[0]'
        center_txt = self.__position_text(center_blk_name)

        normal_blk_name = self.blk_names + '  normal_vector[0]'
        normal_txt = self.__position_text(normal_blk_name)
                
        surf_math = ""
        for idir in range(len(self.math_v)):
            if(float(normal_txt[idir]) == 0.0):
                continue
            else:
                surf_math = surf_math + '(' + normal_txt[idir] + ')'
                if(float(center_txt[idir]) == 0.0):
                    surf_math = surf_math + self.math_v[idir]
                else:
                    surf_math = surf_math + '[' + self.math_v[idir] + ' - (' + center_txt[idir] + ')]'
            if(idir == 2):
                surf_math = surf_math + ' = 0'
            else:
                surf_math = surf_math + ' + '
        return surf_math

    def __ellipsoid_equation_text(self):
        center_blk_name = self.blk_names + '  center_position[0]'
        center_txt = self.__position_text(center_blk_name)

        axis_blk_name = self.blk_names + '  axial_length[0]'
        axis_txt = self.__position_text(axis_blk_name)
                
        surf_math = ""
        for idir in range(len(self.math_v)):
            if(float(axis_txt[idir]) == 0.0):
                continue
            else:
                surf_math = surf_math + '\\frac{'
                if(float(center_txt[idir]) == 0.0):
                    surf_math = surf_math + self.math_v[idir] + '^{2}}'
                else:
                    surf_math = surf_math + '[' + self.math_v[idir] + ' - (' + center_txt[idir] + ')]^{2}}'
                surf_math = surf_math + '{(' + axis_txt[idir] + ')^{2}}'
            if(idir == 2):
                surf_math = surf_math + ' = 0'
            else:
                surf_math = surf_math + ' + '
        return surf_math

    
    def surface_coef_editbox(self):
        self.Tx =          self.make_editbox(self.blk_names)
        self.math_button = self.make_update_button(self.blk_names)
        self.math_button.on_click(self.update_surface_eq_pressed)
        display(self.surface_eq_output, self.Tx, self.math_button, self.update_output)
        self.update_surface_equation()

    def update_surface_equation(self):
        method_label = 'section_method'

        eq_flag =        'equation'
        plane_flag =     'plane'
        sphere_flag =    'sphere'
        ellipsoid_flag = 'ellipsoid'

        self.surface_eq_output.clear_output()

        line_range = self.begin_and_end_block(self.datalist, self.blk_names)
        for index in range(line_range[0]+1,line_range[1]):
            linetext = self.datalist[index]
            if(len(linetext) < 2):
                continue

            label = linetext.split(None)[0]
            mode =  linetext.split(None)[1]

            if(label.upper() == method_label.upper()):
                if(mode.upper() == eq_flag.upper()):
                    iflag_suef_def = 1
                elif(mode.upper() == plane_flag.upper()):
                    iflag_suef_def = 2
                elif(mode.upper() == sphere_flag.upper()):
                    iflag_suef_def = 3
                elif(mode.upper() == ellipsoid_flag.upper()):
                    iflag_suef_def = 4
                break
            
            print('mode', iflag_suef_def, mode)

        if(iflag_suef_def == 1):
            surf_math = self.__surface_equation_text()
        elif(iflag_suef_def == 2):
            surf_math = self.__plane_equation_text()
            print('surf_math', surf_math)
        elif(iflag_suef_def == 3):
            surf_math = self.__sphere_equation_text()
        elif(iflag_suef_def == 4):
            surf_math = self.__ellipsoid_equation_text()
        
        with self.surface_eq_output:
            print("************  Surface equation  **************")
            display(Math(surf_math))
            print("To reload coefs block, press \"shift\" + \"return\".")

        

    def update_surface_eq_pressed(self,*args):
        self.update_block()
        self.update_surface_equation()

#### Routins 6 (Includes all special editors, Final preparation)

Make classes to choose block editors by block name.

- `[Widget_instance] = Control_Editor([Instance], [Block_name])`

This command opens a text editor for block [Block_name] in the control file data [Instance] which is read from control file. After editind the block, update is applied when Update [Block_name] bottun is clicked.  The updated control data around the block are also displayed after clicking the bottun. [Widget_instance] name is only used to make the editor.

In the cases when there is more than one block with the same name, `[Block_name]` can specify the belonged blocks sepalated by space. For example, the folloing command opens editor for `phys_values_ctl` in `model` in `MHD_control`.

`a = Control_Editor(ctl, 'MHD_control model phys_values_ctl')`

If the target block is the array block, Add '[0]' after the array block name. For example, the folloing command opens editor for array `volume_spectrum_ctl` in `sph_monitor_ctl`.

`b = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[0]')`

To specify i-th (i start from 1) block in array block set [i] after the block name. For example, the folloing command opens editor for 2nd block of `volume_spectrum_ctl` in `sph_monitor_ctl`.

`c = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[2]')`

If the target (the last) block name is `dimensionless_ctl`, `coefficients_ctl`, `momentum`, `induction`, `thermal`, `composition`, or `surface_define`, special block editor for the corresponding block is opened to check equaitions.


In [8]:
class Control_Editor(Four_eqs_Editor, Surface_define_editor):
    def __init__(self, control, block_names):
        self.datalist = control.datalist
        self.blk_names = block_names
        
        self.update_output = widgets.Output()
        self.heat_output = widgets.Output()
        self.comp_output = widgets.Output()
        self.mom_output = widgets.Output()
        self.mag_output = widgets.Output()
        self.dimless_output = widgets.Output()        
        self.surface_eq_output = widgets.Output()

        self.Kemo_eq_editbox()

        
    def Kemo_eq_editbox(self):
        dless_block =   'dimensionless_ctl'
        mom_block =     'momentum'
        mag_block =     'induction'
        heat_block =    'thermal'
        comp_block =    'composition'
        four_eq_block = 'coefficients_ctl'

        psf_define_block = 'surface_define'
        
        tgt_block = self.find_target_block_name(self.blk_names)
        
        if(tgt_block.upper() == dless_block.upper()):
            self.dimensionless_editbox()
        elif(tgt_block.upper() == mom_block.upper()):
            self.momentum_eq_editbox()
        elif(tgt_block.upper() == mag_block.upper()):
            self.induction_eq_editbox()
        elif(tgt_block.upper() == heat_block.upper()):
            self.heat_eq_editbox()
        elif(tgt_block.upper() == comp_block.upper()):
            self.composition_eq_editbox()
        elif(tgt_block.upper() == four_eq_block.upper()):
            self.four_eq_editbox()
            
        elif(tgt_block.upper() == psf_define_block.upper()):
            self.surface_coef_editbox()
        
        else:
            self.block_editbox()

## Read control file

Read control file and store each line in list. In the following boxes, move the directory with 'control_MHD', check files in the directory, and read the file.  

[Instance] = Control_Obj()  
[Instance].read_control_file([file_name])


These routines initialize instance of the object for control data and read control file [file_name] and store the text data into class [Instance].

In this section, Following processes take to save the file.
1. Check current directory
2. Move to directory to save
3. Check files in the moved directory
4. Open control file 'control_MHD'

In [9]:
pwd

'/Volumes/Sources/matsui/Calypso/src/Jupyter'

In [10]:
cd Test

/Volumes/Sources/matsui/Calypso/src/Jupyter/Test


In [11]:
ls -l

total 60
-rw-r--r--@ 1 matsui  staff  13479 Aug 10 08:21 control_MHD
-rw-r--r--  1 matsui  staff    644 Aug  9 17:22 control_iso_temp
-rw-r--r--  1 matsui  staff    697 Aug  9 17:22 control_psf_CMB
-rw-r--r--  1 matsui  staff    704 Aug  9 17:22 control_psf_ICB
-rw-r--r--  1 matsui  staff   1153 Aug  9 17:22 control_psf_eq
-rw-r--r--  1 matsui  staff   1179 Aug  9 17:22 control_psf_s0.55
-rw-r--r--  1 matsui  staff   1184 Aug  9 17:22 control_psf_z0.3
-rw-r--r--  1 matsui  staff   1202 Aug  9 17:22 control_psf_zm
-rw-r--r--  1 matsui  staff    665 Aug  9 17:22 control_resolution
-rw-r--r--  1 matsui  staff  10271 Aug  9 17:22 control_snapshot


In [12]:
ctl = Control_Obj()
ctl.read_control_file('control_MHD')

File ' control_MHD ' is loaded.
control_MHD  has  380 lines.


In [13]:
print(ctl.file_name)

control_MHD


## Make new control data (Please skip if control file is read)

This commmands are used for control data from scratch. 

[Instance] = Control_Obj()  
[Instance].new_control_data([block_name])

These routines initialize instance of the object for control data and make control data with empty block named [block_name] into class [Instance].

In this example, new cotrol data with empty `MHD_control` block is made in the instance `ctl2`.

In [14]:
ctl2 = Control_Obj()
ctl2.new_control_data('MHD_control')

******* New control data  *********
0 begin MHD_control
1 end MHD_control


## Write control file

Write edited control file. If output file is exist, program does not write file with message.  
  
[Instance].write_control_file([file_name])

This routine write control file text in [Instance] to file [file_name].

In this section, Following processes take to save the file.
1. Check current directory
2. Move to directory to save
3. Check files in the moved directory
4. Save the control data into 'control_MHD3'

In [6]:
pwd

'/Users/matsui/Desktop/tako'

In [5]:
cd /Users/matsui/Desktop/tako

/Users/matsui/Desktop/tako


In [4]:
ls -l

total 620
-rw-r--r--   1 matsui  staff   81147 Aug  9 11:47 Calypso_control_editor.ipynb
-rw-r--r--   1 matsui  staff   91461 Aug  9 11:45 Calypso_control_glossary.ipynb
-rw-r--r--@  1 matsui  staff    1901 Jul 27 19:54 Folding_Calypso.vim
-rw-------@  1 matsui  staff   17040 Jun 23 16:42 control_MHD
-rw-------@  1 matsui  staff    1106 Jun 26 23:05 control_resolution
drwxr-xr-x@ 30 matsui  staff     960 Aug  8 08:55 controls/
-rw-r--r--   1 matsui  staff    1955 Aug  9 10:55 just_colorbar.png
-rw-r--r--   1 matsui  staff  172108 Aug  9 10:55 test_block_editing.ipynb
-rw-r--r--@  1 matsui  staff  176191 Aug  9 10:55 test_block_editing_z.ipynb
-rw-r--r--@  1 matsui  staff   29480 Jul 31 16:12 test_block_editor_3.ipynb


In [17]:
file_name = 'control_MHD3'
ctl.write_control_file(file_name)

File  control_MHD3 exists. No overwrite


## List control blocks

In this section, list the blocks in the control data in `ctl` by  

`a = Show_Block_Names(ctl, "")`  

Line #, lebel of block, and text including the block name.

In [52]:
a = Show_Block_Names(ctl, "")

Line#, block level, text
   0  1 [0, 380] begin MHD_control
  18  2   begin data_files_def
  54  2   file spherical_shell_ctl  'control_resolution'
  56  2   begin model
  72  3     begin phys_values_ctl
  73  4       array nod_value_ctl
  99  3     begin time_evolution_ctl
 100  4       array time_evo_ctl
 109  3     begin boundary_condition
 110  4       array bc_temperature
 115  4       array bc_velocity
 120  4       array bc_magnetic_field
 130  3     begin forces_define
 131  4       array force_ctl
 145  3     begin dimensionless_ctl
 146  4       array dimless_ctl
 177  3     begin coefficients_ctl
 178  4       begin thermal
 179  5         array coef_4_termal_ctl
 183  5         array coef_4_t_diffuse_ctl
 189  4       begin momentum
 190  5         array coef_4_velocity_ctl
 194  5         array coef_4_press_ctl
 198  5         array coef_4_v_diffuse_ctl
 202  5         array coef_4_buoyancy_ctl
 208  5         array coef_4_Coriolis_ctl
 213  5         array coef_4_Lorentz_

## Control block editor
Here is the section for block editor. You can move this near the description of each block.  

- `[Widget_instance] = Control_Editor([Instance], [Block_name])`

This command opens a text editor for block [Block_name] in the control file data [Instance] which is read from control file. After editind the block, update is applied when `Update [Block_name]` bottun is clicked. The updated control data around the block are also displayed after clicking the bottun.  

- When `shift + return` keys are pressed in the editor window, the current control block data is reloaded with **discurding changes without push the `Update` Button**.

- When more than one editor for the same or including block are opend, 
the updated change in one editor **does not** automatically applied to the other editors. Please reload the latest control data by pressing `shift + return` in the other editors.

In the cases when there is more than one block with the same name, `[Block_name]` can specify the belonged blocks sepalated by space. For example, 

`a = Control_Editor(ctl, 'MHD_control model phys_values_ctl')`

If the target block is the array block, Add '[0]' after the array block name. Here is the example:

`b = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[0]')`

To specify i-th (i start from 1) block in array block set [i] after the block name as:

`c = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[2]')`

If blank text is specified for [Block_name], the editor includes entire control data.

`d = Control_Editor(ctl, '')`

If the target (the last) block name is `dimensionless_ctl`, `coefficients_ctl`, `momentum`, `induction`, `thermal`, `composition`, or `surface_define`, special block editor for the corresponding block is opened to check equaitions.

In [14]:
a = Control_Editor(ctl, 'phys_values_ctl')

To reload current block, press "shift" + "return".


Textarea(value='    begin phys_values_ctl\n      array nod_value_ctl\n        nod_value_ctl  velocity         …

Button(button_style='success', description='Update phys_values_ctl', layout=Layout(height='40px', width='auto'…

Output()

In [15]:
b = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[0]')

To reload current block, press "shift" + "return".


Textarea(value="    array volume_spectrum_ctl\n      begin volume_spectrum_ctl\n        volume_average_prefix …

Button(button_style='success', description='Update volume_spectrum_ctl[0]', layout=Layout(height='40px', width…

Output()

In [83]:
c = Control_Editor(ctl, 'sph_monitor_ctl volume_spectrum_ctl[2]')

To reload current block, press "shift" + "return".


Textarea(value="      begin volume_spectrum_ctl\n        volume_average_prefix        'monitor/sph_ave_inner_c…

Button(button_style='success', description='Update volume_spectrum_ctl[2]', layout=Layout(height='40px', width…

Output()

And, Special editor for governing equations will open by choosing 'coefficients_ctl' block.

In [84]:
d=Control_Editor(ctl,'MHD_control model coefficients_ctl')

Output()

Output()

Output()

Output()

Output()

Textarea(value='    begin coefficients_ctl\n      begin thermal\n        array coef_4_termal_ctl\n          co…

Button(button_style='success', description='Update Equations', style=ButtonStyle())

Output()

Array coef_4_composit_buoyancy_ctl  is missing.
Array coef_4_heat_source_ctl  is missing.
Array coef_4_composition_ctl  is missing.
Array coef_4_c_diffuse_ctl  is missing.
Array coef_4_light_source_ctl  is missing.


Another control file can be loaded into the different instance of `Control_Obj`.
In this example, a control file for cross sectioning is loading into instance `sec1`

In [81]:
sec1 = Control_Obj()
sec1.read_control_file('control_psf_ICB')
a = Show_Block_Names(sec1, "")

File ' control_psf_ICB ' is loaded.
control_psf_ICB  has  30 lines.
Line#, block level, text
   4  1 [4, 30] begin cross_section_ctl
   7  2   begin surface_define
  10  3     array center_position
  18  3     array section_area_ctl
  23  2   begin output_field_define
  24  3     array output_field


And, new control editor for `surface_define` block opens here.

In [82]:
z = Control_Editor(sec1, "surface_define")

Output()

Textarea(value='  begin surface_define\n    section_method    sphere\n!\n    array center_position\n      cent…

Button(button_style='success', description='Update surface_define', layout=Layout(height='40px', width='auto')…

Output()